In [16]:
import json
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn import random_projection
import numpy as np
import random
from scipy.spatial import distance

workdir = Path("/home/studyztp/test_ground/experiments/hardware-profiling/NPB_protocol/NPB3.4.2/NPB3.4-OMP")

In [17]:
# Random Seed at file level
random_seed = 627

np.random.seed(random_seed)
random.seed(random_seed)

benchmark = ["bt", "cg", "ep", "ft", "is", "lu", "mg", "sp"]

In [18]:
def find_rep_rid(data, labels, centers):
    rep_rid = {}
    for i, center in enumerate(centers):
        min = float('inf')
        min_rid = -1
        for j, label in enumerate(labels):
            if label == i:
                dist = distance.euclidean(center, data[j])
                if dist < min and j != 0 and j != 1:
                    min = dist
                    min_rid = j
        rep_rid[i] = min_rid

    return rep_rid

In [19]:
def clustering(data, k):
    all_clusters = {}
    for i in range(10, k+1, 10):
        kmeans = KMeans(n_clusters=i, random_state=random_seed)
        kmeans.fit(data)

        centers = kmeans.cluster_centers_
        labels = kmeans.labels_
        inertia = kmeans.inertia_
        n_iter = kmeans.n_iter_
        rep_rid = find_rep_rid(data, labels.tolist(), centers.tolist())

        all_clusters[i] = {
            "centers": centers.tolist(),
            "labels": labels.tolist(),
            "inertia": inertia,
            "n_iter": n_iter,
            "rep_rid": rep_rid
        }

    return all_clusters


In [20]:
with open(workdir/"basic_block_matrix_info.json", "r") as f:
    basic_block_matrix_info = json.load(f)

all_clustering_data = {}

for bench in benchmark:
    data = basic_block_matrix_info[bench]["matrix"]
    data = np.array(data)
    transformer = random_projection.SparseRandomProjection(15)
    data = transformer.fit_transform(data)
    all_clustering_data[bench] = clustering(data, 30)

with open(workdir/"clustering_data.json", "w") as f:
    json.dump(all_clustering_data, f, indent=4)

    

/home/studyztp/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/studyztp/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/studyztp/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/studyztp/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to

In [21]:
with open(Path(workdir/"marker_info.json"), "r") as f:
    marker_info = json.load(f)

In [23]:
bash_script = ""
import shutil
for bench in benchmark:
    bench_dir = Path(workdir/bench.upper())
    cluster_dir = Path(bench_dir/"clusters")
    if cluster_dir.exists():
        shutil.rmtree(cluster_dir)
    cluster_dir.mkdir(exist_ok=False)

    clusters = all_clustering_data[bench][10]["rep_rid"]
    for rid in clusters.values():
        print(f"Writing down {rid} for {bench}")
        region_markers = marker_info[bench][str(rid)]
        writedown = str(region_markers["warmup_marker_function_id"]) + "\n" \
                +  str(region_markers["warmup_marker_bbid"]) + "\n" \
                +  str(region_markers["warmup_marker_count"]) + "\n" \
                +  str(region_markers["start_marker_function_id"]) + "\n" \
                +  str(region_markers["start_marker_bbid"]) + "\n" \
                +  str(region_markers["start_marker_count"]) + "\n" \
                +  str(region_markers["end_marker_function_id"]) + "\n" \
                +  str(region_markers["end_marker_bbid"]) + "\n" \
                +  str(region_markers["end_marker_count"]) +"\n"
        file_path = Path(cluster_dir/f"{rid}.txt")
        with open(cluster_dir/f"{rid}.txt", "w") as f:
            f.write(writedown)
        bash_script += f"REGION={rid}\n"
        bash_script += f"cp {file_path.as_posix()} /home/studyztp/stuffs/info.txt\n"
        bash_script += f"cd {bench_dir.as_posix()} && "
        bash_script += "make m5_fs REGION=${REGION}\n"
        bash_script += f"cd {bench_dir.as_posix()} && "
        bash_script += "make papi REGION=${REGION}\n"

with open(workdir/"run_script.sh", "w") as f:
    f.write(bash_script)




Writing down 5600 for bt
Writing down 4156 for bt
Writing down 8893 for bt
Writing down 7745 for bt
Writing down 4680 for bt
Writing down 7993 for bt
Writing down 10760 for bt
Writing down 9548 for bt
Writing down 10764 for bt
Writing down 6773 for bt
Writing down 200 for cg
Writing down 5272 for cg
Writing down 3539 for cg
Writing down 3753 for cg
Writing down 3608 for cg
Writing down 5128 for cg
Writing down 3329 for cg
Writing down 3033 for cg
Writing down 4117 for cg
Writing down 3681 for cg
Writing down 138 for ep
Writing down 636 for ep
Writing down 251 for ep
Writing down 282 for ep
Writing down 624 for ep
Writing down 616 for ep
Writing down 625 for ep
Writing down 25 for ep
Writing down 399 for ep
Writing down 488 for ep
Writing down 930 for ft
Writing down 2312 for ft
Writing down 3 for ft
Writing down 3452 for ft
Writing down 2638 for ft
Writing down 193 for ft
Writing down 2 for ft
Writing down 3299 for ft
Writing down 2477 for ft
Writing down 3461 for ft
Writing down 56 fo